# YOLOv11 Poo Detection Pipeline - Complete Implementation

This notebook implements the complete pipeline for training a YOLOv11 poo detection model optimized for mobile deployment using LiteRT.

## Pipeline Overview:
1. **Project Setup**: Clone repository and install dependencies
2. **Dataset Download**: Download and extract semantic_masks dataset
3. **Data Preprocessing**: Convert semantic masks to YOLO format
4. **Model Training**: Train YOLOv11 model with customizable parameters
5. **Model Evaluation**: Evaluate performance and visualize results
6. **LiteRT Export**: Export optimized model for Android deployment

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sciber/deepoo-yolo11-litert/blob/main/notebooks/complete_pipeline.ipynb)

## 1. Configuration and Setup

In [ ]:
import os
import sys
import subprocess
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import display, Image as IPImage

# Pipeline Configuration - Customize as needed
CONFIG = {
    # Project settings
    'project_name': 'deepoo-yolo11-litert',
    'project_url': 'https://github.com/your-username/deepoo-yolo11-litert.git',
    'base_dir': '/content' if 'google.colab' in sys.modules else '.',
    
    # Training settings
    'model_variant': 'yolo11n.pt',  # yolo11n.pt, yolo11s.pt, yolo11m.pt, yolo11l.pt, yolo11x.pt
    'epochs': 100,
    'imgsz': 640,
    'batch_size': -1,  # Auto batch size
    'patience': 50,
    
    # Evaluation settings
    'visualize_samples': 10,
    'conf_threshold': 0.25,
    'iou_threshold': 0.45,
    
    # Export settings
    'quantize': True,
    'optimize': True,
    'simplify': True,
}

print("Pipeline Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

### Clone Repository and Install Dependencies

In [ ]:
# Set up project directory
project_dir = Path(CONFIG['base_dir']) / CONFIG['project_name']

# Clone repository if not exists
if not project_dir.exists():
    print("Cloning repository...")
    subprocess.run(['git', 'clone', CONFIG['project_url'], str(project_dir)], check=True)
    print("Repository cloned successfully!")
else:
    print("Repository already exists.")

# Change to project directory
os.chdir(project_dir)
print(f"Working directory: {os.getcwd()}")

# Install dependencies
print("\nInstalling dependencies...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], check=True)
print("Dependencies installed successfully!")

## 2. Dataset Setup

In [ ]:
# Create data directory structure
data_dir = Path('data')
dataset_path = data_dir / 'semantic_masks'

# Create dataset structure
(dataset_path / 'images').mkdir(parents=True, exist_ok=True)
(dataset_path / 'masks').mkdir(parents=True, exist_ok=True)

print(f"Dataset structure created at: {dataset_path}")
print("\nNote: Please place your semantic_masks dataset in the data/semantic_masks directory")
print("Expected structure:")
print("  data/semantic_masks/")
print("  ├── images/  # Original images from cameras A & B")
print("  └── masks/   # Corresponding bitmap masks")

# Verify dataset
images_dir = dataset_path / 'images'
masks_dir = dataset_path / 'masks'
image_count = len(list(images_dir.glob('*')))
mask_count = len(list(masks_dir.glob('*')))
print(f"\nDataset verification: {image_count} images, {mask_count} masks")

## 3. Data Preprocessing

In [ ]:
# Run preprocessing
print("Starting data preprocessing...")
start_time = datetime.now()

try:
    result = subprocess.run([sys.executable, 'src/data/preprocess.py'], 
                          capture_output=True, text=True, check=True)
    
    print("✅ Preprocessing completed successfully!")
    print(f"Duration: {datetime.now() - start_time}")
    
    if result.stdout:
        print("\nOutput:", result.stdout)
        
except subprocess.CalledProcessError as e:
    print(f"❌ Preprocessing failed: {e}")
    if e.stderr:
        print(f"Error: {e.stderr}")

## 4. Model Training

In [ ]:
# Training configuration
Path('models').mkdir(exist_ok=True)

train_cmd = [
    sys.executable, 'src/models/train.py',
    '--data', 'data/boxed_640x640/dataset.yaml',
    '--output', 'models/',
    '--model', CONFIG['model_variant'],
    '--epochs', str(CONFIG['epochs']),
    '--imgsz', str(CONFIG['imgsz']),
    '--batch', str(CONFIG['batch_size']),
    '--patience', str(CONFIG['patience'])
]

print(f"Training command: {' '.join(train_cmd)}")
print(f"\nStarting training ({CONFIG['epochs']} epochs)...")

training_start = datetime.now()

try:
    result = subprocess.run(train_cmd, capture_output=True, text=True, check=True)
    
    print(f"\n✅ Training completed successfully!")
    print(f"Duration: {datetime.now() - training_start}")
    
    if result.stdout:
        print("\nTraining output:", result.stdout[-1000:])  # Last 1000 chars
        
except subprocess.CalledProcessError as e:
    print(f"❌ Training failed: {e}")
    if e.stderr:
        print(f"Error: {e.stderr}")

## 5. Model Evaluation

In [ ]:
# Check if model exists
model_path = Path('models/best.pt')
if not model_path.exists():
    print(f"❌ Model not found at {model_path}")
else:
    print(f"✅ Model found: {model_path} ({model_path.stat().st_size / (1024*1024):.2f} MB)")
    
    # Run evaluation
    eval_cmd = [
        sys.executable, 'src/models/eval.py',
        '--model', str(model_path),
        '--data', 'data/boxed_640x640/dataset.yaml',
        '--visualize', str(CONFIG['visualize_samples']),
        '--conf', str(CONFIG['conf_threshold']),
        '--iou', str(CONFIG['iou_threshold'])
    ]
    
    print("\nStarting evaluation...")
    
    try:
        result = subprocess.run(eval_cmd, capture_output=True, text=True, check=True)
        
        print("✅ Evaluation completed successfully!")
        if result.stdout:
            print("\nEvaluation results:", result.stdout)
            
    except subprocess.CalledProcessError as e:
        print(f"❌ Evaluation failed: {e}")

## 6. LiteRT Export

In [ ]:
# Export to LiteRT
if model_path.exists():
    export_cmd = [
        sys.executable, 'src/models/export_litert.py',
        '--model', str(model_path),
        '--output', 'models/',
        '--imgsz', str(CONFIG['imgsz'])
    ]
    
    if CONFIG['quantize']:
        export_cmd.extend(['--quantize', '--data', 'data/boxed_640x640/dataset.yaml'])
    if CONFIG['optimize']:
        export_cmd.append('--optimize')
    if CONFIG['simplify']:
        export_cmd.append('--simplify')
    
    print(f"Export command: {' '.join(export_cmd)}")
    print("\nStarting LiteRT export...")
    
    try:
        result = subprocess.run(export_cmd, capture_output=True, text=True, check=True)
        
        print("✅ Export completed successfully!")
        if result.stdout:
            print("\nExport results:", result.stdout)
            
        # Check for exported model
        tflite_files = list(Path('models').glob('*.tflite'))
        if tflite_files:
            for tflite_file in tflite_files:
                size_mb = tflite_file.stat().st_size / (1024*1024)
                print(f"\n📱 Exported model: {tflite_file} ({size_mb:.2f} MB)")
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Export failed: {e}")
else:
    print("❌ Cannot export - model not found")

## 7. Pipeline Summary

In [ ]:
# Pipeline completion summary
print("🎉 Pipeline Execution Summary")
print("=" * 50)

# Check preprocessing
processed_dir = Path('data/boxed_640x640')
if processed_dir.exists():
    print("✅ Data preprocessing: COMPLETED")
    for split in ['train', 'val', 'test']:
        count = len(list((processed_dir / 'images' / split).glob('*.jpg')))
        print(f"   {split}: {count} images")
else:
    print("❌ Data preprocessing: FAILED")

# Check training
if model_path.exists():
    print(f"✅ Model training: COMPLETED ({model_path.stat().st_size / (1024*1024):.2f} MB)")
else:
    print("❌ Model training: FAILED")

# Check evaluation
eval_dir = Path('data/evaluation/predictions')
if eval_dir.exists():
    vis_count = len(list(eval_dir.glob('**/*_pred.jpg')))
    print(f"✅ Model evaluation: COMPLETED ({vis_count} visualizations)")
else:
    print("❌ Model evaluation: FAILED")

# Check export
tflite_files = list(Path('models').glob('*.tflite'))
if tflite_files:
    total_size = sum(f.stat().st_size for f in tflite_files) / (1024*1024)
    print(f"✅ LiteRT export: COMPLETED ({len(tflite_files)} models, {total_size:.2f} MB total)")
    for f in tflite_files:
        print(f"   📱 {f.name} ({f.stat().st_size / (1024*1024):.2f} MB)")
else:
    print("❌ LiteRT export: FAILED")

print("\n🚀 Ready for Android integration!")
print("Next steps:")
print("1. Copy .tflite file to Android app assets")
print("2. Use TensorFlow Lite Android API")
print("3. Input: RGB images 640x640, normalized [0,1]")
print("4. Output: YOLO detection format")